In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc 

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
spark

In [4]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType
from pyspark.ml import PipelineModel
from pyspark.sql.functions import udf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from pyspark.ml import PipelineModel

globals()['models_loaded'] = False
globals()['my_model'] = None

globals()['model_path'] = '/Users/bijiben/Desktop/BDA/advancedanalytics/assignment3/spark/notebooks/CV_pipelineFit.model'

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to DataFrame
    df = spark.read.json(rdd)
    df.show()
    
    # Load the model if not yet loaded
    if not globals()['models_loaded']:
        globals()['my_model'] = PipelineModel.load(globals()['model_path'])
        globals()['models_loaded'] = True
        
    # Apply the loaded model to the DataFrame
    df_result = globals()['my_model'].transform(df)
    df_result = df_result.select("app_id", "review_id", "label","review_id","review_text", "rawPrediction", "probability", "prediction")
    df_result.show()    

In [5]:
ssc = StreamingContext(sc, 10)

In [6]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

23/05/19 18:00:26 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:26 WARN BlockManager: Block input-0-1684512026000 replicated to only 0 peer(s) instead of 1 peers
23/05/19 18:00:27 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:27 WARN BlockManager: Block input-0-1684512027000 replicated to only 0 peer(s) instead of 1 peers
23/05/19 18:00:29 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:29 WARN BlockManager: Block input-0-1684512029000 replicated to only 0 peer(s) instead of 1 peers


23/05/19 18:00:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:31 WARN BlockManager: Block input-0-1684512031000 replicated to only 0 peer(s) instead of 1 peers


========= 2023-05-19 18:00:30 =========


23/05/19 18:00:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:33 WARN BlockManager: Block input-0-1684512033000 replicated to only 0 peer(s) instead of 1 peers


23/05/19 18:00:37 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:37 WARN BlockManager: Block input-0-1684512037000 replicated to only 0 peer(s) instead of 1 peers
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2311190|    1|138547745|its good. but i c...|
|1557990|    1|138546188|       I Love Hitler|
|1940340|    1|138549443|        Game is sick|
+-------+-----+---------+--------------------+

23/05/19 18:00:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:39 WARN BlockManager: Block input-0-1684512039000 replicated to only 0 peer(s) instead of 1 peers


23/05/19 18:00:41 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:41 WARN BlockManager: Block input-0-1684512041000 replicated to only 0 peer(s) instead of 1 peers
23/05/19 18:00:43 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:00:43 WARN BlockManager: Block input-0-1684512043000 replicated to only 0 peer(s) instead of 1 peers


+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+
| app_id|review_id|label|review_id|         review_text|       rawPrediction|         probability|prediction|
+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+
|2311190|138547745|    1|138547745|its good. but i c...|[-0.7943495097714...|[0.31123550916578...|       1.0|
|1557990|138546188|    1|138546188|       I Love Hitler|[-2.2719431902061...|[0.09347342367079...|       1.0|
|1940340|138549443|    1|138549443|        Game is sick|[-1.8225252120493...|[0.13913114334360...|       1.0|
+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+

========= 2023-05-19 18:00:40 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1940340|    1|138548210|This game is good...|
|

========= 2023-05-19 18:00:50 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
| 758690|    1|138548170|Good game but nee...|
| 758690|    1|138547786|   THEN GOOD IS GOOD|
|2268000|    1|138549910|The game is all a...|
+-------+-----+---------+--------------------+

+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+
| app_id|review_id|label|review_id|         review_text|       rawPrediction|         probability|prediction|
+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+
| 758690|138548170|    1|138548170|Good game but nee...|[-0.8438211866585...|[0.30073060876475...|       1.0|
| 758690|138547786|    1|138547786|   THEN GOOD IS GOOD|[-2.2115620568588...|[0.09871700677654...|       1.0|
|2268000|138549910|    1|138549910|The game is all a...|[-0.8775821011032...|[0.

23/05/19 18:01:13 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:01:13 WARN BlockManager: Block input-0-1684512073200 replicated to only 0 peer(s) instead of 1 peers
23/05/19 18:01:14 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/19 18:01:14 WARN BlockManager: Block input-0-1684512074200 replicated to only 0 peer(s) instead of 1 peers


In [8]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
23/05/19 18:01:16 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at org.apache.spark.streaming.dstream.SocketReceiver$$an

Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja

========= 2023-05-19 18:01:20 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2369390|    0|138549797|Bottom List of th...|
|2369390|    1|138549203|Awesome Game play...|
+-------+-----+---------+--------------------+

+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+
| app_id|review_id|label|review_id|         review_text|       rawPrediction|         probability|prediction|
+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+
|2369390|138549797|    0|138549797|Bottom List of th...|[-1.8225252120493...|[0.13913114334360...|       1.0|
|2369390|138549203|    1|138549203|Awesome Game play...|[-1.8225252120493...|[0.13913114334360...|       1.0|
+-------+---------+-----+---------+--------------------+--------------------+--------------------+----------+

